In [1]:
import os
from scipy.interpolate import interp1d
import pandas as pd
import plotly.express as px
import numpy as np
import geopy
import colorcet as cc

import airbnb_dataset
import config_data

COVID_DATA_DIR = './COVID-19Data/Data/'

In [ ]:
myABData = airbnb_dataset.AirbnbDataset()
ctry_cities = list(config_data.country_to_city.values())
state_cities = []
states = []
for k,v in config_data.US_state_to_city.items():
    state_cities.extend(v)
    for t in range(len(v)):
        states.append(k)
print(list(zip(state_cities,states)))

In [ ]:
#need to process reviews in a datewise manner
# one csv per month, across all locations
alldata=[]
cols = ["city","lat","long"]
for i in range(1,11):
    cols.append("month"+str(i))
print(cols)

In [ ]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent='myapplication')
def getlocation(city):
    location = geolocator.geocode(city)
    print(location)
    ns1 = float(location.raw['boundingbox'][0])
    ns2 = float(location.raw['boundingbox'][1])
    ew1 = float(location.raw['boundingbox'][2])
    ew2 = float(location.raw['boundingbox'][3])
    
    return (ns1+ns2)/2,(ew1+ew2)/2

In [ ]:
location = geolocator.geocode("Twin Cities Minneapolis")
print(location)

In [ ]:
for i,c in enumerate(ctry_cities):
    df = myABData.get_reviews_for_city(c,per_week=False,per_month=True)
    df19 = df.loc[2019].values[:-2]
    df20 = df.loc[2020].values
    rel_diff = [c]
    if c == 'Twin Cities MSA':
        c = c[:c.rindex(c)]
    
    lat,long = getlocation(c)#+' ' + states[i]) #only for state level
    rel_diff.append(lat)
    rel_diff.append(long)
    rel_diff.extend(list(((df19 - df20)/df19).T[0]))
    alldata.append(tuple(rel_diff))
monthlydf = pd.DataFrame(alldata, columns=cols)
monthlydf = monthlydf.set_index(["city"])
monthlydf.to_csv(os.path.join(config_data.AIRBNB_DATA_DIR,'GLOBAL/monthly_reviews1.csv'))

In [ ]:
#weekly, 10 months
alldata = []
cols = ["city","lat","long"]

min_n_months = 1000
for i,c in enumerate(ctry_cities):
    df = myABData.get_reviews_for_city(c,per_week = True,per_month = False)
    df20 = df.loc[2020].values[:43]
    df19 = df.loc[2019].values[:43]
    rel_diff = [c]
    if c == 'Twin Cities MSA':
        c = c[:c.rindex(' ')]
    lat,long = getlocation(c)# + " "+states[i])
    rel_diff.append(lat)
    rel_diff.append(long)
    rel_diff.extend(list(((df19 - df20)/df19).T[0]))
    alldata.append(tuple(rel_diff))
for i in range(1,44):
    cols.append("week"+str(i))
weeklydf = pd.DataFrame(alldata,columns = cols)
weeklydf.to_csv(os.path.join(config_data.AIRBNB_DATA_DIR,'GLOBAL/weekly_reviews1.csv'))

In [27]:
#GLOBAL
cols = []
for i in range(1,44):
    cols.append("week"+str(i))
df = pd.read_csv(os.path.join(config_data.AIRBNB_DATA_DIR,'GLOBAL/weekly_reviews1.csv'))


minval = 1000
for col in cols:
    list1 = df[col].values.tolist()
    minval = min(minval,min(list1))

for i,col in enumerate(cols):
    list1 = df[col].values.tolist()
    list1 = list(np.array(list1) - minval)
    m = interp1d([0,5],[1,80])
    circ_rad = m(list1)
    fig = px.density_mapbox(df, title=f'Worldwide - Week {i},2020',color_continuous_scale=px.colors.sequential.Reds, lat="lat", lon="long",center={'lat':17,'lon':25},radius=circ_rad,zoom=1,mapbox_style="open-street-map",opacity=0.85,width=1000,height=700)
    #orange is most affected area
    fig.write_image(os.path.join(config_data.AIRBNB_DATA_DIR,f'GLOBAL/{col}.png'))

In [26]:
#US
color_list = [ 'rgb(39,100,25)', 'rgb(77,146,33)','rgb(127,188,65)','rgb(253,174,97)', 'rgb(244,109,67)', 'rgb(215,48,39)', 'rgb(165,0,38)']

cols = []
for i in range(1,42):
    cols.append("week"+str(i))
df = pd.read_csv(os.path.join(config_data.AIRBNB_DATA_DIR,'US/weekly_reviews1.csv'))
minval = 1000
ttl = 'United States - Week ,2020'
for col in cols:
    list1 = df[col].values.tolist()
    minval = min(minval,min(list1))
    
for i,col in enumerate(cols[12:]):
    print(i)
    list1 = df[col].values.tolist()
    list1 = list(np.array(list1) - minval)
    m = interp1d([0,4],[1,80])
    circ_rad = m(list1)
    to_df = []
    for j in range(len(list1)):
        lati = df["lat"].values.tolist()[j]
        long = df["long"].values.tolist()[j]
        to_df.append((list1[j],lati,long))
    df2=pd.DataFrame(to_df, columns = ["val", "lat", "long"])
    fig = px.density_mapbox(df2,title=f'United States - Week {i},2020',color_continuous_scale=px.colors.sequential.Reds, lat="lat", lon="long",center={'lat':34,'lon':-110},radius=circ_rad,zoom=2.6,mapbox_style="open-street-map",opacity=0.85,width=1000,height=700)
    #orange is most affected area
#     fig.write_image(os.path.join(config_data.AIRBNB_DATA_DIR,f'US/{col}.png'))
    fig.show()
    break

0


In [10]:
px.colors.sequential.Reds
print(px.colors.diverging.PiYG)

color_list = [ 'rgb(184,225,134)', 'rgb(127,188,65)', 'rgb(77,146,33)', 'rgb(39,100,25)','rgb(239,59,44)',
 'rgb(203,24,29)',
 'rgb(165,15,21)',
 'rgb(103,0,13)']


['rgb(142,1,82)', 'rgb(197,27,125)', 'rgb(222,119,174)', 'rgb(241,182,218)', 'rgb(253,224,239)', 'rgb(247,247,247)', 'rgb(230,245,208)', 'rgb(184,225,134)', 'rgb(127,188,65)', 'rgb(77,146,33)', 'rgb(39,100,25)']


In [17]:
# https://towardsdatascience.com/four-ways-to-quantify-synchrony-between-time-series-data-b99136c4a9c9
# cross correlation between the number of COVID-19 Cases and Number of Reviews dropped from last year
print(px.colors.diverging.RdYlBu[::-1])

['rgb(49,54,149)', 'rgb(69,117,180)', 'rgb(116,173,209)', 'rgb(171,217,233)', 'rgb(224,243,248)', 'rgb(255,255,191)', 'rgb(254,224,144)', 'rgb(253,174,97)', 'rgb(244,109,67)', 'rgb(215,48,39)', 'rgb(165,0,38)']
